In [1]:
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score
import sys

sys.path.append("../")

import abus_classification

In [2]:
dataset = abus_classification.datasets.TDSCTumors(path="../data/tdsc")    

In [3]:
import multiprocessing



def extract_signature(dataset):
    signatures = []
    for i in range(len(dataset)):
        _,x,y = dataset[i]
        sig = abus_classification.utils.features.boundary_signature_3d(x, resolution=(5,5))
        signatures.append((sig.flatten(), y))
        
    return signatures

num_processes = multiprocessing.cpu_count() - 2
chunk_size = len(dataset)//num_processes
data_chunks = [[dataset[j] for j in range(i, i+chunk_size)] for i in range(0, len(dataset), chunk_size)]

pool = multiprocessing.Pool(processes=num_processes)

signature_dataset = pool.map(extract_signature, data_chunks)

pool.close()
pool.join()


: 

In [46]:
num_processes = multiprocessing.cpu_count()
print(num_processes)

12


In [22]:
X, Y = zip(*signature_dataset)
X = list(X)
Y = list(Y)

In [23]:
def classify_with_svm(x, y):
    
    acc = 0
    train_acc = 0
    cfm = [[0,0],
           [0,0]]

    for i in range(100):
        x_test, y_test = x[i], y[i]
        X_train, Y_train = x[:i] + x[i+1:], y[:i] + y[i+1:]
        clf = SVC()
        clf.fit(X_train, Y_train)
        P = clf.predict(X_train)
        train_acc += accuracy_score(Y_train, P)
        res = clf.predict([x_test])[0]
        if res == y_test:
            acc += 1
            
        cfm[y_test][res] += 1
            
    print(f"Train accuracy: {train_acc/100}")
    print(f"Accuracy: {acc/100}")
    print(f"{cfm[0]}\n{cfm[1]}")

In [28]:
classify_with_svm(X,Y)

Train accuracy: 0.7259595959595961
Accuracy: 0.68
[44, 14]
[18, 24]


In [35]:
def normalize_(x:np, interval=(0,1)):
    x_std = (x - x.min())/x.max()-x.min()
    mi, ma = interval
    return x_std*(ma-mi) + mi

In [36]:
def normalize(x):
    for i in range(100):
        x[i] = normalize_(x[i])
    return x


In [39]:
X = normalize(X)
classify_with_svm(X,Y)

Train accuracy: 0.726767676767677
Accuracy: 0.65
[50, 8]
[27, 15]


In [42]:
signature_dataset = []

for i in tqdm(range(len(dataset))):
    _,x,y = dataset[i]
    sig = abus_classification.utils.features.boundary_signature_3d(x, resolution=(20,20))
    signature_dataset.append((sig.flatten(), y))
    
X, Y = zip(*signature_dataset)
X = list(X)
Y = list(Y)

  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
classify_with_svm(X,Y)
X = normalize(X)
classify_with_svm(X,Y)

Train accuracy: 0.7139393939393939
Accuracy: 0.69
[45, 13]
[18, 24]
Train accuracy: 0.7110101010101009
Accuracy: 0.68
[54, 4]
[28, 14]


In [ ]:
import torch
import torch.nn as nn

class GRUClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(GRUClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Forward propagate GRU
        out, _ = self.gru(x, h0)  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

# Example usage
input_size = 1  # Assuming each element in the sequence is represented by a single feature
hidden_size = 64
num_layers = 1
num_classes = 2  # Two classes for classification

# Initialize the model
model = GRUClassifier(input_size, hidden_size, num_layers, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Assuming you have data loader `train_loader` containing training data
# Assuming each batch of training data is of shape (batch_size, seq_length, input_size)
# And `labels` containing corresponding labels

# Training loop
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (sequences, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(sequences)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Once trained, you can use the model for predictions
# For example, to predict on a single sequence:
sequence_to_predict = torch.randn(1, 360, input_size)  # Example random sequence
output = model(sequence_to_predict)
predicted_class = torch.argmax(output, dim=1)
print("Predicted class:", predicted_class.item())
